# Chosen Cyphertext Attack

## Libraries

In [1]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from mod import Mod
from Crypto.Util.number import GCD

## Getting the modulo

### The Math


For pairs of plaintext and ciphertext expressed as pi and ci, the following holds
> $$p_i^e = c_i \mod{n}$$
where e is the encryption key and n is the modulus of the RSA

Rearanging it gets us
> $$p_i^e - c_i = x_i n$$
where $x_i n$ is a multiple of the modulus

Such that taking 2 pairs $i$ and $j$ where $i \gt j$, getting the GCD of the difference between the plaintext to the power of $e$ and the ciphertext will yield $n$ as long as the GCD of $x_i$ and $x_j$ are 1
> if $GCD(x_i, x_j) = 1$ then:
> $$GCD(p_i^e - c_i, p_j^e - c_j) = n$$

### The Process

We know that n is a 2048 bit integer, so assuming e is a random integer between $2^{15}$ and $2^{16}$, we can attempt a brute force attack on the n. Heuristically, there is a very low chance that the first GCD of the pairs is less than 10000, so we accept 10000 as a minimum to filter out the negative results.

In [2]:
def getn(p1, c1, p2, c2):

    it = range(2**15, 2**16 + 1)
    results = []
    with tqdm(total=len(it)) as pbar:
        def calcGCD(e):
            pair1 = pow(p1, e) - c1
            pair2 = pow(p2, e) - c2
            xn = GCD(pair1, pair2)
            pbar.update(1)
            if (xn > 10000):
                return xn
            else:
                return None

        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(calcGCD, e) for e in it]
            for future in as_completed(futures):
                if future.result() is not None:
                    results.append(future.result())

    return results

There is no easy way to get n exactly, so we are hoping to get lucky such that $GCD(x_i, x_j) = 1$, we will now choose two plaintexts that are close to each other 2 and 3 as the base for this attack

In [3]:
p1 = 2
c1 = 85955117279290968705782214705802035051623686453430194318885811595524534807295104846928423484162054301322669005675171398331297946067618660213209992648752960362284958423128161004628150057868695882803520368868439791989427748010559105270954204026941195766363602533717119220827012683892344025587109176523229330234

In [4]:
p2 = 3
c2 = 58540332411930629350296896968112777697810274133009925988527571443385139001449898333915697772112681776437372258753322519444425464391900655051123918914592391586694255761158489422123769886435000979709721866972738220608783329904775425529175194169940882074117650849391411706039619199247214940657210001898561446134

In [5]:
n = getn(p1, c1, p2, c2)
print(n)

100%|██████████| 32769/32769 [04:28<00:00, 121.98it/s]

[94688288693672755703811561595239067912943346941373520038546059177837194924230262112364763158713690534133531340332148132862841318454370901485464294816638255978670260687066798369477551761808746099256605106814688867177276597580972721175909942368919953717356720594127161731307445778299631169385948728154159008987]


In [6]:
n = max(n)
print(n)

94688288693672755703811561595239067912943346941373520038546059177837194924230262112364763158713690534133531340332148132862841318454370901485464294816638255978670260687066798369477551761808746099256605106814688867177276597580972721175909942368919953717356720594127161731307445778299631169385948728154159008987


After Getting the n, we heuristically try to remove any small prime factors to increase the success chance

In [7]:
for i in range(2, 100000):
    if Mod(n, i) == 0:
        n //= i

print(n)

94688288693672755703811561595239067912943346941373520038546059177837194924230262112364763158713690534133531340332148132862841318454370901485464294816638255978670260687066798369477551761808746099256605106814688867177276597580972721175909942368919953717356720594127161731307445778299631169385948728154159008987


## Getting the Ciphertext of the admin plaintext

### The Math

Assuming two of the following equations
> $$p^e = c \mod n$$
and
> $$c^d = p \mod n$$

for a plaintext $p$ which is a multiple of two plaintexts, plaintext factors $p_i$ and $p_j$,
> $$p = p_i \times p_j$$
the ciphertext is
> $$\lparen p_i \times p_j \rparen^e = c \mod n$$
> $$p_i^e \times p_j^e = c\mod n$$
> $$c_i \times c_j = c\mod n$$


Thus we just need to know two ciphertexts of two plaintexts that when multiplied together equal the desired plaintext and we can multiply them together in modulo n to get the desired ciphertext

### The Process

In [8]:
p = 41176563

In [9]:
pi = p2
ci = c2

In [10]:
pj = p // pi
print(pj)

13725521


In [11]:
cj = 16229219966305127509788787076967563075693493583479123753686782953880201648394294921628309132183923886144666077713063298093787304145549887716570003624020380543444644097895525983783175144986033633497999041781185786978346892775414838300818428474842187090590771161967982774403363302225479286439061370972428541216

In [12]:
c = int(Mod(ci * cj, n))
print(c)

63012758239432692119792292683979194532583557245607065200039400230021977692233450148180780334960275294751002675901230008982324846250878200784783046053205681176457011631652332337031611285003614817949864921258202639492263758678735087795160712761383945889934838121365836501196139120986372392715033290881871749367
